# s1300012_exercise13

In [ ]:
!pip install PAMI

## Data preprocessing

In [ ]:
import pandas as pd
from PAMI.extras.DF2DB import denseDF2DB as pro

# データの読み込み
file_path = '/content/air_pollution_data.csv'  # Google Driveに保存したファイルのパス
dataset = pd.read_csv(file_path)

# データの前処理
dataset.drop(columns=['Timestamp'], inplace=True)
dataset = dataset.fillna(0)
dataset.where(dataset <= 100, 0, inplace=True)

# トランザクションデータベースの作成
db = pro.denseDF2DB(inputDF=dataset, thresholdValue=15, condition='>=')

# トランザクションデータベースの保存
output_file_path = '/content/PM24HeavyPollutionRecordingSensors.csv'
db.createTransactional(outputFile=output_file_path)

# 保存した CSV ファイルの確認
trans_df = pd.read_csv(output_file_path)
print(trans_df.head())


## Knowledge discovery

In [ ]:
from PAMI.frequentPattern.basic import FPGrowth as alg

# 入力ファイルの指定
inputFile = 'PM24HeavyPollutionRecordingSensors.csv'

# モデルに使用する制約条件の指定
minSup = 1

# マイニングアルゴリズムのオブジェクトを作成
obj = alg.FPGrowth(inputFile, minSup)

# マイニングプロセスを開始
obj.startMine()

# 生成されたパターンをファイルに保存
obj.save('frequentPatterns.txt')


######  Frequent patterns were generated successfully using frequentPatternGrowth algorithm
##  
##  Visualization

In [ ]:
import re
import plotly.express as px
import pandas as pd

# Read the file to get a pattern
with open('frequentPatterns.txt', 'r') as file:
    patterns = file.readlines()

# Analyze the pattern to identify the longest pattern
max_pattern = []
max_length = 0
for pattern in patterns:
    # Obtain coordinates in the pattern
    coordinates = re.findall(r'Point\((.*?)\)', pattern)
    if len(coordinates) > max_length:
        max_pattern = coordinates
        max_length = len(coordinates)

# Display the longest pattern on Plotly Express Open Street Map
df = pd.DataFrame(max_pattern, columns=['coordinate'])
df[['Lon', 'Lat']] = df['coordinate'].str.split(' ', expand=True)
df['Lon'] = df['Lon'].astype(float)
df['Lat'] = df['Lat'].astype(float)
fig = px.scatter_mapbox(
    df,
    lat='Lat',
    lon='Lon',
    hover_name='coordinate'
)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

[リンクテキスト](https://github.com/s1300012/PAMI/blob/main/notebooks/Exercise13.ipynb)